In [1]:
import numpy as np
from scipy import sparse
import pickle as pkl

In [4]:
# Needs: 
# pseudotime trajectory
# eigenvalues as function of time
# gexp for neutrophil genes
# gexp for myelocite genes


headdir    = '/Users/sfreedman/Code/sc_bifurc_figs'
figdir     = '{0}/figs'.format(headdir)
datdir     = '{0}/neutrophil_data'.format(headdir)
eigdir     = '{0}/eig'.format(datdir)
#eigdir     = '{0}/eig_ncell_sample'.format(datdir)

gexp_fname = '{0}/gene_expr.npz'.format(datdir)
pst_fname  = '{0}/pseudotime.txt'.format(datdir)
gnm_fname  = '{0}/gene_names.txt'.format(datdir)
meta_fname = '{0}/metadata.txt'.format(datdir)

# In[6]:

print('loading gene expression matrix')
gexp_sp    = sparse.load_npz(gexp_fname) # WT: 18.3 seconds
gexp_lil   = gexp_sp.tolil() # WT: 3 min 55 seconds


# In[8]:

print('loading cluster labels and SPRING positions')
dtp      = np.dtype([('Library Cell', np.unicode_, 16),('barcode', np.unicode_, 20),
              ('Time point', int),('Starting population', np.unicode_, 20),
               ('Cell type annotation', np.unicode_, 60),
               ('Well', int), ('SPRING-x', np.float64), ('SPRING-y', np.float64)])

metadata = np.genfromtxt(meta_fname, delimiter='\t',skip_header=1, dtype=dtp)

nms      = dtp.names
gnms     = np.genfromtxt(gnm_fname,dtype='str')

print('loading neutrophil pseudotime ranking')
neut_psts = np.genfromtxt(pst_fname, skip_header=True, dtype='int')


# In[12]:

print('binning gene expression')
bin_sz            = 1000
overlap           = int(bin_sz/2)

srt               = np.argsort(neut_psts[:,1])
last_full_bin     = int(np.floor(srt.shape[0]/overlap)*overlap) - bin_sz + overlap
neut_pst_grps     = [srt[i:(i+bin_sz)] for i in range(0,last_full_bin,overlap)]
neut_pst_grps[-1] = np.union1d(neut_pst_grps[-1], srt[last_full_bin:])
neut_pst_cidxs    = [np.array(neut_psts[grp,0], dtype = 'int') for grp in neut_pst_grps]
npsts             = len(neut_pst_cidxs)

loading gene expression matrix
loading cluster labels and SPRING positions
loading neutrophil pseudotime ranking
binning gene expression


In [5]:
###############################################################
# marker genes
###############################################################

# In[26]:

print('marker gene expression')
gene_group_labs  = ['neutrophil','MPP','GPP','PMy','My']

neut_gnms        = np.array(['S100a9', 'Itgb2l', 'Elane', 'Fcnb', 'Mpo', 'Prtn3', 
                              'S100a6', 'S100a8', 'Lcn2', 'Lrg1'])
mpp_gnms         = np.array(['Ly6a','Meis1','Flt3','Cd34'])
gmp_gnms         = np.array(['Csf1r','Cebpa'])
pmy_gnms         = np.array(['Gfi1','Elane'])
my_gnms          = np.array(['S100a8','Ngp','Ltf'])

grp_gnms  = [neut_gnms, mpp_gnms, gmp_gnms, pmy_gnms, my_gnms]

grp_gidxs = [np.array([np.where(gnms==gnm)[0][0] for gnm in k]) for k in grp_gnms]


###############################################################
# additional highly varying, highly expressed genes...#
###############################################################
print('matrix of gene expression for highly expressed / varying genes')

nnz_thresh  = 0
cv_thresh   = 0.5
gexp_thresh = 1

mu_gexp = np.array([np.mean(gexp_lil[cidxs].toarray(),axis=0) for cidxs in neut_pst_cidxs]) # takes like a minute
std_gexp = np.array([np.std(gexp_lil[cidxs].toarray(),axis=0) for cidxs in neut_pst_cidxs]) # takes like a minute

ng           = mu_gexp.shape[1]
nnzs         = np.sum(mu_gexp>0,axis=0)
mu_mu_gexp   = np.mean(mu_gexp,axis=0)
max_mu_gexp  = np.max(mu_gexp,axis=0)

std_mu_gexp  = np.std(mu_gexp,axis=0)
cvs          = np.divide(std_mu_gexp, mu_mu_gexp, out = np.zeros(ng), where=mu_mu_gexp>0)

hi_gidxs        = np.where((nnzs > nnz_thresh) & (cvs>cv_thresh) & (max_mu_gexp>gexp_thresh))[0]

marker gene expression
matrix of gene expression for highly expressed / varying genes


In [6]:
gexp_full = gexp_sp.toarray() # less than a minute

In [7]:
#dnb_gidxs       = np.union1d(np.hstack(grp_gidxs), hi_gidxs)
dnb_gidxs       = np.unique(np.hstack(grp_gidxs))
non_dnb_gidxs   = np.setdiff1d(np.arange(ng),dnb_gidxs)
#np.save('{0}/high_var_gexp_trajs.npy'.format(datdir), mu_gexp)

In [8]:
def dnb_biomarker(t): # takes about 9 seconds per 1000 cell group
#t = 0   
    gexp     = gexp_full[neut_pst_cidxs[t]]
    gexp_std = np.std(gexp,axis  = 0)
    nz_gidxs = np.where(gexp_std > 0)[0]
    gcorr    = np.corrcoef(gexp[:,nz_gidxs].T)

    nz_dnb_gidxs, dnb_corr_gidxs,_ = np.intersect1d(nz_gidxs, dnb_gidxs, assume_unique = True, 
                                                    return_indices=True)
    _,non_dnb_corr_gidxs,_         = np.intersect1d(nz_gidxs, non_dnb_gidxs, assume_unique = True, 
                                                    return_indices=True)

    dnb_corrs = np.array([gcorr[dnb_corr_gidxs[i],dnb_corr_gidxs[j]] 
                       for i in range(len(dnb_corr_gidxs)) 
                       for j in range(i+1,len(dnb_corr_gidxs))])

    non_dnb_corrs = np.array([gcorr[non_dnb_corr_gidxs[i],non_dnb_corr_gidxs[j]] 
                       for i in range(len(dnb_corr_gidxs)) 
                       for j in range(len(non_dnb_corr_gidxs))])

    dnb_mu_std          = np.mean(gexp_std[nz_dnb_gidxs]) 
    dnb_mu_abs_corr     = np.mean(np.abs(dnb_corrs))
    non_dnb_mu_abs_corr = np.mean(np.abs(non_dnb_corrs))
    return dnb_mu_std * dnb_mu_abs_corr / non_dnb_mu_abs_corr

In [9]:
import time
t=time.time()
t2=time.time()
print(t2-t)

8.702278137207031e-05


In [10]:
nsamp           = 100
nt              = len(neut_pst_cidxs)
n_dnb_gidxs     = len(np.hstack(grp_gidxs))
dnb_gidxs_rand  = np.array([np.random.choice(ng, n_dnb_gidxs, replace=False) for i in range(nsamp)])
dnb_biom_rand   = np.zeros((nsamp,nt))

for t in range(nt):
    t0 = time.time()

    gexp     = gexp_full[neut_pst_cidxs[t]]
    gexp_std = np.std(gexp,axis  = 0)
    nz_gidxs = np.where(gexp_std > 0)[0]
    gcorr    = np.corrcoef(gexp[:,nz_gidxs].T)
    
    for i in range(nsamp):
        dnb_gidxs      = dnb_gidxs_rand[i]
        non_dnb_gidxs = np.setdiff1d(np.arange(ng),dnb_gidxs)

        nz_dnb_gidxs, dnb_corr_gidxs,_ = np.intersect1d(nz_gidxs, dnb_gidxs, assume_unique = True, 
                                                        return_indices=True)
        _,non_dnb_corr_gidxs,_         = np.intersect1d(nz_gidxs, non_dnb_gidxs, assume_unique = True, 
                                                        return_indices=True)

        dnb_corrs = np.array([gcorr[dnb_corr_gidxs[i],dnb_corr_gidxs[j]] 
                           for i in range(len(dnb_corr_gidxs)) 
                           for j in range(i+1,len(dnb_corr_gidxs))])

        non_dnb_corrs = np.array([gcorr[non_dnb_corr_gidxs[i],non_dnb_corr_gidxs[j]] 
                           for i in range(len(dnb_corr_gidxs)) 
                           for j in range(len(non_dnb_corr_gidxs))])

        dnb_mu_std          = np.mean(gexp_std[nz_dnb_gidxs]) 
        dnb_mu_abs_corr     = np.mean(np.abs(dnb_corrs))
        non_dnb_mu_abs_corr = np.mean(np.abs(non_dnb_corrs))
        dnb_biom_rand[i,t]  = dnb_mu_std * dnb_mu_abs_corr / non_dnb_mu_abs_corr
        
    print('t={0} took {1:.2f} seconds'.format(t, time.time()-t0))


t=0 took 9.48 seconds
t=1 took 9.75 seconds
t=2 took 9.84 seconds
t=3 took 9.81 seconds
t=4 took 10.52 seconds
t=5 took 9.90 seconds
t=6 took 9.88 seconds
t=7 took 10.44 seconds
t=8 took 10.11 seconds
t=9 took 10.75 seconds
t=10 took 10.01 seconds
t=11 took 9.60 seconds
t=12 took 10.10 seconds
t=13 took 9.81 seconds
t=14 took 10.77 seconds
t=15 took 9.70 seconds
t=16 took 9.57 seconds
t=17 took 9.80 seconds
t=18 took 10.77 seconds
t=19 took 10.12 seconds
t=20 took 10.80 seconds
t=21 took 10.31 seconds
t=22 took 10.48 seconds
t=23 took 10.35 seconds
t=24 took 10.96 seconds
t=25 took 10.25 seconds
t=26 took 10.02 seconds
t=27 took 10.44 seconds
t=28 took 11.11 seconds
t=29 took 10.48 seconds
t=30 took 10.57 seconds
t=31 took 11.21 seconds
t=32 took 10.95 seconds
t=33 took 10.43 seconds
t=34 took 10.33 seconds
t=35 took 11.16 seconds
t=36 took 10.76 seconds
t=37 took 11.33 seconds
t=38 took 10.96 seconds
t=39 took 10.96 seconds
t=40 took 10.97 seconds
t=41 took 11.34 seconds
t=42 took 11.

In [11]:
np.save('{0}/pst_dnb_random.npy'.format(datdir), dnb_biom_rand)

In [12]:
nt = len(neut_pst_cidxs)
dnb_biom = np.zeros(nt)
for t in range(nt):
    if t%10==0:
        print('t={0}'.format(t))
    dnb_biom[t] = dnb_biomarker(t)

t=0
t=10
t=20
t=30
t=40
t=50
t=60
t=70
t=80
t=90
t=100
t=110
t=120


In [13]:
save_fnm = 'pst_dnb_nnz_{0}_cv{1}_gexp{2}'.format(nnz_thresh, cv_thresh, gexp_thresh).replace('.','-')
save_fnm = 'pst_dnb_markers'
np.save('{0}/{1}.npy'.format(datdir, save_fnm), dnb_biom)

In [14]:
datdir,save_fnm

('/Users/sfreedman/Code/sc_bifurc_figs/neutrophil_data', 'pst_dnb_markers')

In [ ]:
/Users/sfreedman/Code/sc_bifurc_figs//neutrophil_data/pst_dnb_nnz_0_cv0-5_gexp1.npy'

In [117]:
dnb_biom

array([2.51528798, 2.35548517, 2.2993535 , 2.25606962, 2.27464471,
       2.19634677, 2.05366841, 2.11306998, 2.14854462, 2.09241615,
       2.03708608, 2.05362581, 2.04288023, 2.14068559, 2.10054504,
       2.10317786, 2.252113  , 2.20858826, 2.22966654, 2.28285245,
       2.20783458, 2.18293207, 2.18427446, 2.12272778, 2.10426362,
       2.12987264, 2.17004066, 2.26828752, 2.21605136, 2.19277449,
       2.22277477, 2.23170421, 2.38079216, 2.3680383 , 2.42260137,
       2.52814381, 2.53975657, 2.39560726, 2.35381244, 2.46570855,
       2.48106105, 2.48452592, 2.70546443, 2.71194899, 2.58533405,
       2.59650589, 2.81084784, 2.81422695, 2.59406872, 2.63806423,
       2.7472489 , 2.74705939, 2.95163497, 2.91402806, 2.80524596,
       2.88889808, 2.90068405, 3.04320318, 3.05833159, 2.79142373,
       2.88003509, 2.98640256, 2.9041966 , 2.92770407, 3.03419307,
       3.12115733, 2.90598638, 2.95755718, 3.00499049, 2.95547043,
       2.93628941, 3.00240662, 2.96289463, 2.85387258, 2.92072

In [ ]:
fig,axs=plt.subplots()

In [25]:
#gexp_corr0 = np.corrcoef(gexp_lil[neut_pst_cidxs[0]].T)

In [14]:
type(gexp_lil[neut_pst_cidxs[0]])

scipy.sparse.lil.lil_matrix

In [15]:
def sparse_corr(A):
    N = A.shape[0]
    C=((A.T*A -(sum(A).T*sum(A)/N))/(N-1)).todense()
    V=np.sqrt(np.mat(np.diag(C)).T*np.mat(np.diag(C)))
    COR = np.divide(C,V+1e-119)
    return COR

In [24]:
%time gexp_corr0_lil = sparse_corr(gexp_lil[neut_pst_cidxs[0]])

CPU times: user 39.1 s, sys: 1min 14s, total: 1min 53s
Wall time: 2min 37s


In [23]:
%time gexp_corr0_csr = sparse_corr(gexp_csr[neut_pst_cidxs[0]])

CPU times: user 41.4 s, sys: 44.5 s, total: 1min 25s
Wall time: 1min 58s


In [20]:
gexp_corr0.shape

(25289, 25289)

In [18]:
type(gexp_sp)

scipy.sparse.coo.coo_matrix

In [21]:
gexp_csr =  gexp_sp.tocsr() #11:48...11:49

In [22]:
gexp_csr.shape

(130887, 25289)

In [28]:
gexp_full.shape

(130887, 25289)

In [30]:
gexp_full[neut_pst_cidxs[0]].shape

(1000, 25289)

In [33]:
%time gexp_corr0_nonsp = np.corrcoef(gexp_full[neut_pst_cidxs[0]].T)

/Users/simonfreedman/opt/anaconda3/envs/bifurc/lib/python3.8/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/simonfreedman/opt/anaconda3/envs/bifurc/lib/python3.8/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


CPU times: user 26.8 s, sys: 4.37 s, total: 31.1 s
Wall time: 25.3 s


In [36]:
gexp_corr0_nonsp.shape

(25289, 25289)

In [37]:
np.where(np.isnan(gexp_corr0_nonsp))[0].shape

AttributeError: 'tuple' object has no attribute 'shape'

In [107]:
%time dnb_biomarker(0)

CPU times: user 8.34 s, sys: 761 ms, total: 9.1 s
Wall time: 7.24 s


2.515287978713839

In [108]:
%time gcorr    = np.corrcoef(gexp[:,nz_gidxs].T)

CPU times: user 6.56 s, sys: 523 ms, total: 7.08 s
Wall time: 5.34 s


In [111]:
%time non_dnb_corrs = np.mean(np.abs(np.array([gcorr[non_dnb_corr_gidxs[i],non_dnb_corr_gidxs[j]] for i in range(len(dnb_corr_gidxs)) for j in range(len(non_dnb_corr_gidxs))])))


CPU times: user 1.81 s, sys: 107 ms, total: 1.91 s
Wall time: 2.06 s


In [84]:
np.amax(dnb_corr_gidxs), gcorr.shape, nz_gidxs.shape, dnb_corr_gidxs.shape

(13565, (13545, 13545), (13545,), (336,))

In [41]:
np.where(np.isnan(gexp_corr0_nonsp))[0].shape

(456066496,)

In [46]:
gexp0 = gexp_full[neut_pst_cidxs[0]]

In [48]:
gexp0.shape

(1000, 25289)

In [58]:
nz_gidxs = np.where(np.sum(gexp0,axis=0)>0)[0]

In [75]:
nz_gidxs2 = np.where(np.std(gexp0,axis=0)>0)[0]

In [63]:
%time gcorr0 = np.corrcoef(gexp0[:,nz_gidxs].T)

CPU times: user 6.93 s, sys: 632 ms, total: 7.56 s
Wall time: 5.48 s


In [69]:
dnb_corr_gidxs     = np.intersect1d(nz_gidxs, dnb_gidxs,     assume_unique = True, return_indices=True)[1]
non_dnb_corr_gidxs = np.intersect1d(nz_gidxs, non_dnb_gidxs, assume_unique = True, return_indices=True)[1]

In [73]:
dnb_gidxs.shape, dnb_corr_gidxs.shape, non_dnb_corr_gidxs.shape

((349,), (336,), (13226,))

In [ ]:
dnb_corrs = np.array([[corrs[t,dnb_corrgenes[i],dnb_genes[j]] 
                       for i in range(len(dnb_genes)) 
                       for j in range(i+1,len(dnb_genes))] 
                      for t in range(nt)])

non_dnb_corrs = np.array([[corrs[t,dnb_genes[i],non_dnb_genes[j]] 
                       for i in range(len(dnb_genes)) 
                       for j in range(len(non_dnb_genes))] 
                       for t in range(nt)])
